In [2]:
!pip install "transformers==4.41.2" "peft==0.11.1" "trl==0.9.4" "datasets==2.20.0" "accelerate==0.31.0" bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: tokenizers
    F

In [8]:
# Final script for Google Colab Environment - AGGRESSIVE OPTIMIZATION

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from peft import LoraConfig
from trl import SFTTrainer



# --- Authentication ---
from huggingface_hub import login
# You will be prompted to enter your token here
login()

# --- Configuration ---
BASE_MODEL = "google/gemma-2b-it"
DATASET_PATH = "dataset.jsonl"
NEW_MODEL_NAME = "gemma-2b-coach"

# --- QLoRA Configuration ---
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# --- Load Resources ---
print("Loading dataset...")
dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

print(f"Loading base model: {BASE_MODEL}")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()},
)

# --- PEFT (LoRA) Configuration - MORE AGGRESSIVE ---
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8, # REDUCED from 16
    bias="none",
    task_type="CAUSAL_LM",
)

# --- Training Configuration ---
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=1, # MINIMUM BATCH SIZE
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
)

# --- Prompt Formatting ---
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"<start_of_turn>user\n{example['instruction'][i]}\n{example['input'][i]}<end_of_turn>\n<start_of_turn>model\n{example['output'][i]}<end_of_turn>"
        output_texts.append(text)
    return output_texts

# --- Trainer Initialization ---
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=512, # REDUCED from 2048
    tokenizer=tokenizer,
    args=training_arguments,
)

# --- Execute Training ---
print("Starting fine-tuning process with aggressive memory optimization...")
trainer.train()

# --- Save Final Asset ---
print(f"Saving trained LoRA adapter to ./{NEW_MODEL_NAME}")
trainer.model.save_pretrained(NEW_MODEL_NAME)

print("Fine-tuning complete. Asset is ready for download.")

Loading dataset...
Loading base model: google/gemma-2b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConf

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Starting fine-tuning process with aggressive memory optimization...


Step,Training Loss
5,3.646900
10,2.940600
15,2.828100
20,2.801600


Saving trained LoRA adapter to ./gemma-2b-coach
Fine-tuning complete. Asset is ready for download.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
!ls -l

total 36
-rw-r--r-- 1 root root 19488 Sep 12 09:41 dataset.jsonl
drwxr-xr-x 2 root root  4096 Sep 12 10:17 gemma-2b-coach
drwxr-xr-x 3 root root  4096 Sep 12 09:46 results
drwxr-xr-x 1 root root  4096 Sep  9 13:46 sample_data
drwxr-xr-x 3 root root  4096 Sep 12 09:49 wandb


In [10]:
!zip -r gemma-2b-coach.zip gemma-2b-coach

  adding: gemma-2b-coach/ (stored 0%)
  adding: gemma-2b-coach/README.md (deflated 66%)
  adding: gemma-2b-coach/adapter_config.json (deflated 51%)
  adding: gemma-2b-coach/adapter_model.safetensors (deflated 9%)
